# Extended Report on the Stellar Community Fund, Round 23

*BlockScience, February 2024* 

## Introduction
In 2023, the Stellar Community Fund (SCF) and BlockScience (BSci) collaborated on ideating and implementing a novel governance mechanism, titled Neural Quorum Governance. Following this initial phase, we are now monitoring and evaluating this mechanism through per-round reports. This allows the community to better inform themselves about the dynamics and effects of this voting mechanism, informing discussions on changes and adaptations. Eventually, the community is expected to take over both the governance mechanism, as well as any analytic means necessary to conduct sustainable and informed community governance. 

Starting with round 22, this report introduces a two part structure through both a descriptive and a counterfactual analysis of each round. While the descriptive part serves to describe and visualize observed outcomes, the counterfactual part serves to inform about "what-ifs". In this second section, we will use our cadCAD model of the NQG mechanism to simulate hypothetical outcomes, such as through adaptation of parameters, active neurons and more. This allows the community to make informed decisions about future changes, before having to implement them in production settings.  


### Terminology Notes
Users can be divided into three categories (per project and round):
* Active Users (actively vote for their own choice)
* Delegating Users (delegate their voting choice to a Quorum of other users)
* Inactive Users (refrain from voting or delegating)

---

## Methodology

This section outlines the methodology employed in integrating user votes, delegation/trust data, reputation, and user voting history data into a cadCAD digital twin simulation, backtesting the data on actual project scores, and running counterfactual scenarios. The study aimed to explore the impact of different voting and governance mechanisms on project outcomes within the Stellar Community Fund (SCF).

### Digital Twin Simulation

1) **Integration of Data into cadCAD** : We utilized the cadCAD (complex adaptive dynamics Computer-Aided Design) simulation environment to create a digital twin of the SCF's voting ecosystem. This involved integrating various datasets:
    - User votes: Records of individual votes cast by users for each project.
    - Delegation/Trust Data: Data representing the delegation of votes and the trust relationships among users.
    - Reputation/ Voting History Data: Historical data of users’ voting behavior within the SCF community.

2) **Modeling of Scenarios with distinct Voting Mechanisms**: The cadCAD model was designed to replicate the NQG (Neural Quorum Governance) along with running counterfactual scenarios for standard voting,  Neural Governance without Quorum Delegation, and Quorum Delegation wihout Neural Governance mechanisms.

### Backtesting with Actual Project Scores
1) **Data Sourcing**: Actual project scores from round 22 were sourced to serve as a benchmark for our simulations.

2) **Backtesting Process**: The integrated cadCAD model was backtested against these actual scores. This process involved running the model with historical data to see how closely the simulation could replicate actual outcomes. This backtesting provided a basis for validating the model's accuracy and reliability.

### Running Counterfactual Scenarios
1) **Scenario Design**: To understand the impact of different governance mechanisms, we designed several counterfactual scenarios:

    - 1 person, 1 vote: Simulating voting outcomes in the absence of the Neural Quorum Governance mechanism assuming 1 person 1 vote.
    - NG without QD: Analyzing the effects of Neural Governance when Quorum Delegation is not a factor by assuming every delegation vote as an abstain vote.
    - QD without NG: Observing outcomes where Quorum Delegation is applied without the influence of Neural Governance.

2) **Comparative Analysis**: The results from these counterfactual scenarios were compared against each other and the backtested results. This comparative analysis aimed to discern the distinct impacts of NQG, NG, and QD on project rankings and community dynamics.


## Loading data

### Imports

In [ ]:
from datetime import datetime
import pandas as pd
import sys
sys.path.append('../')
import plotly.io as pio
pio.renderers.default = "png"


### Reading the voting data into a dataframe

In [ ]:
round_no = 24

from nqg_model.experiment import full_historical_data_run_plus_counterfactual
sim_df, df = full_historical_data_run_plus_counterfactual(folder_path=f'../data/r{round_no}/', round_no=round_no)
# sim_df.to_pickle('../data/r23/sim_df.pkl')

In [ ]:
sim_df.head(3)

In [ ]:
from nqg_model.types import User, ProjectUUID, UserUUID, ActionMatrix, VoteDecisionMatrix, Vote, ProjectAction
from typing import Optional, NamedTuple

def quorum_delegation_results(user: UserUUID,
                              user_quorum: set[UserUUID],
                              project: ProjectUUID,
                              action_matrix: ActionMatrix,
                              vote_decision_matrix: VoteDecisionMatrix) -> Optional[dict[UserUUID | str, str]]:
    

    if action_matrix.get(user, {}).get(project, ProjectAction.Abstain) == ProjectAction.Delegate:
        output = {}
        output[f'Quorum Result'] = vote_decision_matrix[user][project]

        for uid in user_quorum:
            output[uid] = vote_decision_matrix.get(uid, {}).get(project, Vote.Undefined)
        return output
    else:
        return None



row = sim_df.iloc[1]

delegatees = row.delegatees
vote_decision_matrix = row.vote_decision_matrix
action_matrix = row.action_matrix

projects = set()
for vd in vote_decision_matrix.values():
    projects |= set(vd)
projects = list(projects)

user = next(iter(delegatees))
user_quorum = delegatees[user]
project = projects[0]





class QuorumDelegationResultRecord(NamedTuple):
    delegating_user: UserUUID
    submission: ProjectUUID
    delegatee: UserUUID
    outcome: Vote | str

records: list[QuorumDelegationResultRecord] = []

for user, user_quorum in delegatees.items():
    for project in projects:
        results = quorum_delegation_results(user, user_quorum, project, action_matrix, vote_decision_matrix)
        if results is not None:
            new_records = [
                QuorumDelegationResultRecord(user, project, uid, vote.__str__().split(".")[-1]) for uid, vote in results.items()
            ]
            for r in new_records:
                records.append(r)
        else:
            pass

qd_df = pd.DataFrame(records).set_index(['delegating_user', 'submission', 'delegatee'])


qd_df['outcome'] = qd_df.outcome.map({'Yes': "Yes/No", 'No': 'Yes/No', 'Undefined': "Didn't Vote", 'Abstain': 'Abstain'})

In [ ]:
qd_df.head(30)

In [ ]:
qd_df.to_csv(f'../data/r24/sim_results/qd_results_{datetime.now()}.csv')